In [3]:
import torch
import pretty_midi
import numpy as np

# Double Pendulum Energy Loss
def double_pendulum_loss(theta1, theta2, omega1, omega2, torque1, torque2, dt=0.1, m1=1.0, m2=1.0, l1=1.0, l2=1.0, g=9.81):
    delta = theta2 - theta1
    den1 = (m1 + m2)*l1**2 * m2*l2**2 - (m2*l1*l2*torch.cos(delta))**2
    if den1.abs() < 1e-6:
        den1 = torch.tensor(1e-6)
    
    a1_numerator = m2*l2*(l2*omega2**2*torch.sin(delta) - g*torch.sin(theta2)) + (m1 + m2)*g*l1*torch.sin(theta1) + torque1
    a1 = (a1_numerator * m2*l2**2 - m2*l1*l2*torch.cos(delta)*(torque2 - m2*l1*l2*omega1**2*torch.sin(delta))) / den1
    
    a2_numerator = torque2 - m2*l1*l2*(a1 * torch.cos(delta) + omega1**2 * torch.sin(delta)) + m2*g*l2*torch.sin(theta2)
    a2 = a2_numerator / (m2*l2**2)
    
    new_omega1 = omega1 + a1 * dt
    new_omega2 = omega2 + a2 * dt
    new_theta1 = theta1 + new_omega1 * dt
    new_theta2 = theta2 + new_omega2 * dt
    
    ke1 = 0.5 * m1 * (l1 * new_omega1)**2
    v2_sq = (l1 * new_omega1)**2 + (l2 * new_omega2)**2 + 2 * l1 * l2 * new_omega1 * new_omega2 * torch.cos(delta)
    ke2 = 0.5 * m2 * v2_sq
    ke = ke1 + ke2
    
    pe = -m1*g*l1*torch.cos(new_theta1) - m2*g*(l1*torch.cos(new_theta1) + l2*torch.cos(new_theta2))
    
    return ke + pe, new_theta1, new_theta2, new_omega1, new_omega2

# Vibrating String Amplitude Loss
def vibrating_string_loss(u_prev, u_curr, damping_factor, dx=0.1, dt=0.05, c=1.0):
    u_next = torch.zeros_like(u_curr)
    for i in range(1, len(u_curr)-1):
        u_next[i] = 2*u_curr[i] - u_prev[i] + (c*dt/dx)**2 * (u_curr[i+1] - 2*u_curr[i] + u_curr[i-1])
        u_next[i] *= (1 - damping_factor)
    return u_next, torch.abs(u_next[len(u_next)//2]).item()

# Heat Equation Gradient Loss
def heat_equation_loss(temperature, heat_source, dt=0.1, alpha=0.1, dx=0.1):
    next_temp = temperature.clone()
    for i in range(1, len(temperature)-1):
        next_temp[i] = temperature[i] + alpha * dt / dx**2 * (temperature[i+1] - 2*temperature[i] + temperature[i-1])
    next_temp[len(temperature)//2] += heat_source * dt
    gradient = torch.mean(torch.abs(next_temp[1:] - next_temp[:-1]))
    return next_temp, gradient.item()

def main(optimizer_type, num_steps=200):
    # Initialize parameters with requires_grad=True
    torque1 = torch.tensor(0.1, requires_grad=True)
    torque2 = torch.tensor(0.1, requires_grad=True)
    damping = torch.tensor(0.5, requires_grad=True)
    heat_source = torch.tensor(0.1, requires_grad=True)
    
    # Create optimizer
    optimizer = optimizer_type([torque1, torque2, damping, heat_source], lr=0.001)
    
    # Physical system states
    theta1 = torch.tensor(np.pi / 2)
    theta2 = torch.tensor(np.pi / 2)
    omega1 = torch.tensor(0.0)
    omega2 = torch.tensor(0.0)
    u_prev = torch.zeros(10)
    u_curr = torch.zeros(10)
    u_curr[4] = 1.0
    temperature = torch.zeros(10)
    
    # MIDI setup
    midi = pretty_midi.PrettyMIDI()
    piano = pretty_midi.Instrument(program=pretty_midi.instrument_name_to_program('Acoustic Grand Piano'))
    bpm = 120
    current_time = 0.0
    
    for step in range(num_steps):
        optimizer.zero_grad()
        
        # Simulate systems
        energy, new_theta1, new_theta2, new_omega1, new_omega2 = double_pendulum_loss(theta1, theta2, omega1, omega2, torque1, torque2)
        u_next, amplitude = vibrating_string_loss(u_prev, u_curr, torch.sigmoid(damping))
        new_temp, gradient = heat_equation_loss(temperature, heat_source)
        
        # Compute losses
        loss_energy = energy
        loss_amplitude = torch.tensor(amplitude)
        loss_gradient = torch.tensor(gradient)
        
        total_loss = loss_energy + 10 * loss_amplitude + 0.1 * loss_gradient
        total_loss.backward()
        optimizer.step()
        
        # Update states
        theta1, theta2, omega1, omega2 = new_theta1.detach(), new_theta2.detach(), new_omega1.detach(), new_omega2.detach()
        u_prev, u_curr = u_curr.detach(), u_next.detach()
        temperature = new_temp.detach()
        
        # Map to music features
        pitch = int((theta1.item() + theta2.item()) * 10 + 60) % 128
        velocity = int(torch.sigmoid(damping).item() * 127)
        duration = 60.0 / bpm * (1.0 + heat_source.item())
        
        # Add note
        note = pretty_midi.Note(
            velocity=velocity,
            pitch=pitch,
            start=current_time,
            end=current_time + duration
        )
        piano.notes.append(note)
        current_time += duration
    
    midi.instruments.append(piano)
    midi.write('optimizer_music.mid')

if __name__ == "__main__":
    import sys
    from torch.optim import SGD, Adam, RMSprop
    
    main(SGD)

ValueError: cannot convert float NaN to integer